In [ ]:
import os
import numpy as np
import asdf
import json

from glob import glob


# JWST pipeline-related modules
from jwst.datamodels import dqflags

# The entire jwst pipeline
from jwst.pipeline import calwebb_detector1
from jwst.pipeline import calwebb_image2
from jwst.pipeline import calwebb_image3
from jwst import datamodels

# importing an individual pipeline step
from jwst.skymatch import SkyMatchStep

# Custom scripts for use later

import matplotlib.pyplot as plt
from matplotlib import rcParams

# Use this version for non-interactive plots (easier scrolling of the notebook)
#%matplotlib inline

# Use this version if you want interactive plots
#%matplotlib notebook

from astropy.io import fits

%matplotlib inline

# These gymnastics are needed to make the sizes of the figures
# be the same in both the inline and notebook versions
#%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

# You may want to change the following configurations to customize 
# figure sizes and resolutions
rcParams['figure.figsize'] = [11,8]
rcParams['figure.dpi'] = 80
rcParams['savefig.dpi'] = 80

from jwst.associations.asn_from_list import asn_from_list
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base


In [ ]:
os.environ['CRDS_PATH'] = '/Volumes/T7-RED/crds_cache/' 
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'


In [ ]:
import jwst
print(jwst.__version__)

In [ ]:
try:
    print(os.environ['CRDS_PATH'])
except KeyError:
    print('CRDS_PATH environment variable not set!')
    
try:
    print(os.environ['CRDS_SERVER_URL'])
except KeyError:
    print('CRDS_SERVER_URL environment variable not set!')

try:
    print(os.environ['CRDS_CONTEXT'])
except KeyError:
    print('CRDS_CONTEXT environment variable not set!')

In [ ]:
file_id = 'F150W'

proposalID = '2736'

output_dir = file_id+'-calibrated'

print(output_dir)

os.system('mkdir -p {}'.format(output_dir))

# Stage1


In [ ]:
#download the .py files from https://github.com/chriswillott/jwst
import checkifstar
from dosnowballflags import snowballflags


In [ ]:
uncal_file = glob('JWST/*/*uncal.fits')
uncal_file

In [ ]:
uncal_file = glob('JWST/*/*uncal.fits')
uncal_file

for uncal in uncal_file:

# save the jump.fits

    detector1 = calwebb_detector1.Detector1Pipeline()
    detector1.output_dir = '/'.join(uncal.split('/')[0:-1])
    detector1.save_results = True

    detector1.ipc.skip = True

    detector1.jump.save_results = True

    detector1.ramp_fit.skip = True
    detector1.gain_scale.skip = True
    
    run_output = detector1.run(uncal)

# snowball flag to the jump.fits

    jumpdirfile = uncal.replace('uncal','jump')
    imagingmode = True
    filtername = fits.getheader(uncal)['FILTER']
    npixfind = 50
    satpixradius=3
    halofactorradius=2

    snowballflags(jumpdirfile,filtername,npixfind,satpixradius,halofactorradius,imagingmode)

# run the last two step of the pipeline from jump.fits, and save the rate.fits
    detector1 = calwebb_detector1.Detector1Pipeline()
    detector1.output_dir = output_dir
    detector1.save_results = True

    detector1.group_scale.skip = True
    detector1.dq_init.skip = True
    detector1.saturation.skip = True
    detector1.ipc.skip = True
    detector1.superbias.skip = True
    detector1.refpix.skip = True
    detector1.rscd.skip = True
    detector1.firstframe.skip = True
    detector1.lastframe.skip = True
    detector1.linearity.skip = True
    detector1.dark_current.skip = True
    detector1.reset.skip = True
    detector1.persistence.skip = True
    detector1.jump.skip = True

    detector1.ramp_fit.skip = False
    detector1.gain_scale.skip = False

    run_output = detector1.run(jumpdirfile)

#    break




# Stage2


In [ ]:
rates_file = glob(file_id+'-calibrated/*rate.fits')
rates_file

# WISPS subtraction

In [ ]:
#!/usr/bin/env python
import numpy as np
import os
import re
import sys
import astropy.io.fits as fits
from glob import glob
import inspect
#
#==============================================================================
# Wisps are seen in F150W, F150W2, F200W, F210M
# Status unknown in F162M, F182M
# SCAs presenting significant wisps : A3, B3, B4

        
    


In [ ]:

template_dir  = '../../wisp_templates/'
template = dict([
    ('NRCA3_F150W', 'wisps_nrca3_F150W.fits'),
    ('NRCA3_F150W2','wisps_nrca3_F150W2.fits'),
    ('NRCA3_F200W', 'wisps_nrca3_F200W.fits'),
    ('NRCA3_F210M', 'wisps_nrca3_F210M.fits'),
    ('NRCB3_F150W', 'wisps_nrcb3_F150W.fits'),
    ('NRCB3_F150W2','wisps_nrcb3_F150W2.fits'),
    ('NRCB3_F200W', 'wisps_nrcb3_F200W.fits'),
    ('NRCB3_F210M', 'wisps_nrcb3_F210M.fits'),
    ('NRCB4_F150W', 'wisps_nrcb4_F150W.fits'),
    ('NRCB4_F150W2','wisps_nrcb4_F150W2.fits'),
    ('NRCB4_F200W', 'wisps_nrcb4_F200W.fits'),
    ('NRCB4_F210M', 'wisps_nrcb4_F210M.fits')
    ])
    
#
debug  = 0
rate_dir = file_id+'-calibrated/'
files = sorted(glob(rate_dir+'*rate.fits'))
print(files)

for index in range(len(files)):
    file = files[index]
    hdulist = fits.open(file)
    nhdu  = len(hdulist)
    if(debug == 1):
        hdulist.info()

    for ii in range(0, nhdu):
        header = hdulist[ii].header
        if('DETECTOR' in header):
            detector = header['detector']
        if('FILTER' in header):
            filter = header['FILTER']
#    hdulist.close()
            
    if(detector != 'NRCA3' and detector != 'NRCB3' and detector != 'NRCB4'):
        continue

#
# The very few tests carried out in commissioning showed that a
# plain subtraction was effective. If this is not the case, it may
# be necessary toscale the template intensity to determine 
# an optimal subtractionlevel
#
    if(filter == 'F150W' or filter == 'F150W2' or \
       filter == 'F200W' or filter == 'F210M'):

        os.system('cp ' + file + ' ' + file.replace('rate.fits','rate_orig.fits'))

        key = detector+'_'+filter
        print("filter ", filter, 'detector ', detector)
        wisp = template_dir+template[key]
        print("wisp ", wisp)
        median = fits.getdata(wisp,ext=0)
        corrected = hdulist[1].data - median

        corrected[corrected != corrected] = 0

        hdulist[1].data = corrected
        hdulist[1].header['history'] = 'WISPS correction applied'
        new_file = re.sub('rate.fits','rate.fits',file)
        print("org file", file.replace('rate.fits','rate_orig.fits'))
        print("new file", new_file)
        hdulist.writeto(new_file,overwrite=True)

#        os.system('cp ' + new_file + ' ' + new_file.replace('corr','corr_rate'))

    hdulist.close()

#    break



In [ ]:
rates_file = glob(file_id+'-calibrated/*rate.fits')

for rate in rates_file:
    # Create an instance of the pipeline class
    image2 = calwebb_image2.Image2Pipeline()

    # Set some parameters that pertain to the
    # entire pipeline
    image2.output_dir = output_dir
    image2.save_results = True
    # turn off the ResampleStep, comment out to produce the 
    # individual rectified *_i2d.fits for quick-look checks
    image2.resample.skip = True

    # Call the run() method
    image2.run(rate)

In [ ]:
from plotimages import plot_images
from image1overf import sub1fimaging
from astropy.io import fits

In [ ]:
calfiles = glob(output_dir+'/jw*_cal.fits')
calfiles

In [ ]:
output_dir

In [ ]:
for cal2file in calfiles:
    cal21overffile = cal2file.replace('_cal.fits','_cal_1overf.fits')
    print ('Running 1/f correction on {} to produce {}'.format(cal2file,cal21overffile))
    with fits.open(cal2file) as cal2hdulist:
        if cal2hdulist['PRIMARY'].header['SUBARRAY']=='FULL' or cal2hdulist['PRIMARY'].header['SUBARRAY']=='SUB256':
            sigma_bgmask=3.0
            sigma_1fmask=2.0
            splitamps=False   #Set to True only in a sparse field so each amplifier will be fit separately. 
            correcteddata = sub1fimaging(cal2hdulist,sigma_bgmask,sigma_1fmask,splitamps)
            if cal2hdulist['PRIMARY'].header['SUBARRAY']=='FULL':
                cal2hdulist['SCI'].data[4:2044,4:2044] = correcteddata  
            elif cal2hdulist['PRIMARY'].header['SUBARRAY']=='SUB256':
                cal2hdulist['SCI'].data[:252,:252] = correcteddata
            cal2hdulist.writeto(cal21overffile, overwrite=True)

In [ ]:
plot_images(cal21overffile,
            cal2file, 
            title1='1/f corr', title2='cal')

In [ ]:
'JWST-'+file_id

# Stage 3

In [ ]:
calfiles = glob(output_dir+'/jw*_cal_1overf.fits')
asn = asn_from_list(calfiles,rule=DMS_Level3_Base, product_name='test')
asn

In [ ]:
json_stage3 = 'JWST-'+file_id+'-stage3.json'
with open(json_stage3, 'w') as outfile:
    name,serialized = asn.dump(format='json') 
    outfile.write(serialized)

In [ ]:
image3 = calwebb_image3.Image3Pipeline()

# Set some parameters that pertain to the entire pipeline
image3.output_dir = file_id+'-calibrated/'
image3.output_file = file_id+'_i2d.fits'

image3.save_results = True
# Set some parameters that pertain to some of the individual steps

image3.tweakreg.skip = False 
#image3.tweakreg.backend == 'sextractor'
image3.tweakreg.align_to_gaia == True
# Turn off SkyMatchStep

image3.skymatch.subtract = True
image3.skymatch.skip = False
# Set the ratio of input to output pixels to create an output mosaic 
# on a 0.015"/pixel scale

image3.resample.pixel_scale = 0.03
# Call the run() method

image3.run(json_stage3)
